##Importação das bibliotecas



In [0]:
import pandas as pd
import requests
import json
import sqlalchemy as sql

##Leitura do csv para um DataFrame

In [0]:
filiais = pd.read_csv("https://bitbucket.org/somalabs/python-challenge/raw/ee636c254648f5123729337a3815b03aaccdad9e/filiais.csv")

##GET na API em  texto

In [0]:
vendas_api = requests.get("http://private-amnesiac-78bfc-vendas4.apiary-proxy.com/vendas").text

##Transforma o json em um dicionário de python

In [0]:
dic_vendas = json.loads(vendas_api)

##Transforma o dicionário em um Data Frame

In [0]:
df_vendas = pd.DataFrame(dic_vendas)

##Faz o merge das tabelas utilizando como campo em comum o código da filial

In [0]:
vendas_filiais = pd.merge(filiais,df_vendas, left_on = 'codigo_filial', right_on = 'vendido_por')

##Remove a columa redundante

In [0]:
vendas_filiais = vendas_filiais.drop('vendido_por', axis=1)

##Cria o banco de dados

In [0]:
engine = sql.create_engine('sqlite://')
vendas_filiais.to_sql('vendas', con=engine)

##Consulta total de vendas por dia em cada filial

In [0]:
vendas_por_dia = engine.execute("SELECT nome_filial, COUNT(codigo_filial), date(registered) FROM vendas GROUP BY codigo_filial, date(registered)").fetchall()

###Converte saída da consulta para um data frame para melhor visualização

In [10]:
pd.DataFrame(vendas_por_dia, columns = ['Nome filial', 'Quantidade de vendas','Dia'])

,Nome filial,Quantidade de vendas,Dia
0,foxtontijuca,1,2019-01-02
1,foxtontijuca,1,2019-01-05
2,foxtontijuca,1,2019-01-10
3,foxtontijuca,1,2019-01-18
4,foxtontijuca,1,2019-01-29
5,foxtontijuca,1,2019-03-05
6,foxtontijuca,1,2019-03-16
7,foxtontijuca,1,2019-03-19
8,foxtontijuca,1,2019-03-26
9,foxtontijuca,1,2019-04-04


##Consulta informações (nome,total de vendas e valor total em reais) de cada filial

In [0]:
inf_filiais = engine.execute("SELECT nome_filial, COUNT(codigo_filial), SUM(valor_em_centavos)/100.0 FROM vendas GROUP BY codigo_filial").fetchall()

###Converte saida da consulta para um dataframe para a leitura ficar mais clara

In [12]:
pd.DataFrame(inf_filiais, columns = ['Nome filial', 'Quantidade de vendas','Valor total em reais'])

,Nome filial,Quantidade de vendas,Valor total em reais
0,foxtontijuca,19,25817.18
1,foxtongavea,29,47424.73
2,foxtonriosul,23,34315.18
3,foxtonfashionmall,19,30796.57
4,foxtonriodesignbarra,35,58044.13
5,foxtonshoppingleblon,24,35737.27
6,foxtonbarrashopping,30,43192.13
7,foxtonipanema,21,21670.67
